In [6]:
# Import Libraries
import pandas as pd
import folium
import json
import requests
from folium import plugins

# Load the shape of the zone (Vanderburgh County Zip Codes)
with open('TriCountyZip.json', 'r') as f: 
    vanderburgh_geo = f.read()


# Load the ALICE Data for each zone
df = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\TriCounty2016.csv') 
df['ZCTA5CE10'] = df['ZCTA5CE10'].astype(str)

In [7]:
# Initialize Map: 
m = folium.Map(location=[38.01, -87.22], zoom_start=10.35)
m

In [8]:
# Set Up Colormap
from branca.colormap import linear
import branca.colormap as cmp

colormap = cmp.LinearColormap(
    ['blue', 'green', 'yellow', 'orange','red'],
    vmin=10, vmax=70, 
    caption='Percentage of Households Under ALICE Threshold 2016'
)

colormap

In [9]:
#m.save('SWI_Percentage_Households_Below_ALICE Change 2016-2018.html')

In [10]:
df_dict=df.set_index('ZCTA5CE10')['percentALICE']
print(df_dict)

ZCTA5CE10
47708    63.276836
47710    52.945677
47711    37.861586
47712    41.145635
47713    69.041394
47714    54.467009
47715    45.493894
47720    34.260165
47725    11.799801
47523    34.151957
47601    40.093829
47610    38.414942
47613    29.665072
47619    30.136986
47630    28.079332
47637    32.121212
47531    24.860335
47537    20.113314
47550     2.605863
47577    37.606838
47579    23.395445
47588    46.236559
47611    25.190840
47615    39.905363
47634    28.233971
47635    35.521064
Name: percentALICE, dtype: float64


In [16]:
geo_data = json.loads(vanderburgh_geo)
print(geo_data)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'STATEFP10': '18', 'ZCTA5CE10': '47708', 'GEOID10': '1847708', 'CLASSFP10': 'B5', 'MTFCC10': 'G6350', 'FUNCSTAT10': 'S', 'ALAND10': 1435639, 'AWATER10': 5765, 'INTPTLAT10': '+37.9745534', 'INTPTLON10': '-087.5735745', 'PARTFLG10': 'N'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-87.572577, 37.969606], [-87.571894, 37.968764], [-87.572805, 37.968305], [-87.572984, 37.968223], [-87.573178, 37.968472], [-87.573358, 37.968689], [-87.57365, 37.969061], [-87.572577, 37.969606]]], [[[-87.566522, 37.971504], [-87.566413, 37.971358], [-87.565842, 37.970677], [-87.565969, 37.970604], [-87.566768, 37.970196], [-87.56687, 37.970125], [-87.567567, 37.970977], [-87.568641, 37.970433], [-87.569591, 37.96995], [-87.569725, 37.969878], [-87.570397, 37.970707], [-87.57109, 37.97156], [-87.571197, 37.97151], [-87.571297, 37.971455], [-87.571398, 37.971382], [-87.571498, 37.971328], [-87.571544, 37.971309], [-87.57

In [7]:
folium.GeoJson(
    geo_data, 
    style_function = lambda feature:{
        'fillColor':colormap(df_dict[feature['properties']['ZCTA5CE10']]), 
        'color':'black', 
        'weight': 1, 
    }
).add_to(m)
colormap.add_to(m)
m

In [57]:
#m.save("Households Below ALICE Threshold Custom Scalebar 2016.html")

In [58]:
# Initialize Layer Control -> All Markers are one cluster
fg = folium.FeatureGroup(control=False)
m.add_child(fg)

# Import Spreadsheets with Location Information
#locChildCare = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\LicensedDaycares.csv') 
#locLib = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\locationLibraries.csv') 
#locBus = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\METS_Bus_Stops.csv')
#locPark = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\public_parks.csv')
#locSchool = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\school-locations.csv')
childhoodAgn = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\childhood_agencies.csv')
youthAgn = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\youth_agencies.csv')
healthAgn = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\health_agencies.csv')
vitalAgn = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\vital_agencies.csv')
economicAgn = pd.read_csv(r'C:\Users\jhuhnke\Documents\Mapping Alice Data\economic_agencies.csv')

# Initialize Different Sub Layers for Different Types of Markers
#CC = folium.plugins.FeatureGroupSubGroup(fg, 'Child Care Centers')
#m.add_child(CC)

#Lib = folium.plugins.FeatureGroupSubGroup(fg, 'Libraries')
#m.add_child(Lib)

#Bus = folium.plugins.FeatureGroupSubGroup(fg, 'Bus Stops')
#m.add_child(Bus)

#Park = folium.plugins.FeatureGroupSubGroup(fg, 'Parks')
#m.add_child(Park)

#School = folium.plugins.FeatureGroupSubGroup(fg, 'Schools')
#m.add_child(School)

Childhood = folium.plugins.FeatureGroupSubGroup(fg, 'Childhood GRF Agencies')
m.add_child(Childhood)

Youth = folium.plugins.FeatureGroupSubGroup(fg, 'Youth GRF Agencies')
m.add_child(Youth)

Health = folium.plugins.FeatureGroupSubGroup(fg, 'Health GRF Agencies')
m.add_child(Health)

Vital = folium.plugins.FeatureGroupSubGroup(fg, 'Vital GRF Agencies')
m.add_child(Vital)

Economic = folium.plugins.FeatureGroupSubGroup(fg, 'Economic GRF Agencies')
m.add_child(Economic)

# Add Child Care Markers to Map
#for i in range(0, len(locChildCare)):
#    folium.Marker([locChildCare.iloc[i]['lat'], locChildCare.iloc[i]['long']], popup='Licensed Child Care Center', icon=folium.Icon(color='lightgray', icon='child', prefix='fa')).add_to(CC)

# Add Library Markers to Map    
#for i in range(0, len(locLib)): 
#    folium.Marker([locLib.iloc[i]['lat'], locLib.iloc[i]['long']], popup='Library', icon=folium.Icon(color='lightblue', icon='book', prefix='fa')).add_to(Lib)

# Add Bus Stop Markers to Map
#for i in range(0, len(locBus)): 
#    folium.CircleMarker([locBus.iloc[i]['lat'], locBus.iloc[i]['long']], radius=2, popup='Bus Stop').add_to(Bus)

# Add Park Markers to Map
#for i in range(0, len(locPark)): 
#    folium.Marker([locPark.iloc[i]['lat'], locPark.iloc[i]['long']], popup='Park', icon=folium.Icon(color='green', icon='tree', prefix='fa')).add_to(Park)

# Add School Markers to Map
#for i in range(0, len(locSchool)): 
#    folium.Marker([locSchool.iloc[i]['lat'], locSchool.iloc[i]['long']], popup='School', icon=folium.Icon(color='red', icon='graduation-cap', prefix='fa')).add_to(School)
    
# Add Childhood Agency Markers to Map
for i in range(0, len(childhoodAgn)): 
    folium.Marker([childhoodAgn.iloc[i]['lat'], childhoodAgn.iloc[i]['long']], popup='Childhood GRF', icon=folium.Icon(color='red', icon='baby', prefix='fa')).add_to(Childhood)
    
# Add Youth Agency Markers to Map
for i in range(0, len(youthAgn)): 
    folium.Marker([youthAgn.iloc[i]['lat'], youthAgn.iloc[i]['long']], popup='Youth GRF', icon=folium.Icon(color='orange', icon='baby', prefix='fa')).add_to(Youth)

# Add Health Agency Markers to Map
for i in range(0, len(healthAgn)): 
    folium.Marker([healthAgn.iloc[i]['lat'], healthAgn.iloc[i]['long']], popup='Health GRF', icon=folium.Icon(color='green', icon='first-aid', prefix='fa')).add_to(Health)  
    
# Add Vital Agency Markers to Map
for i in range(0, len(vitalAgn)): 
    folium.Marker([vitalAgn.iloc[i]['lat'], vitalAgn.iloc[i]['long']], popup='Vital GRF', icon=folium.Icon(color='blue', icon='star-of-life', prefix='fa')).add_to(Vital)    

# Add Economic Agency Markers to Map
for i in range(0, len(economicAgn)): 
    folium.Marker([economicAgn.iloc[i]['lat'], economicAgn.iloc[i]['long']], popup='Economic Development GRF', icon=folium.Icon(color='purple', icon='money-bill-wave', prefix='fa')).add_to(Economic)    

folium.LayerControl(collapsed=False).add_to(m)
m

In [59]:
m.save("map_layer_selection.html")

In [61]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Marker Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Childhood GRFs</li>
    <li><span style='background:orange;opacity:0.7;'></span>Youth GRFs</li>
    <li><span style='background:green;opacity:0.7;'></span>Health GRFs</li>
    <li><span style='background:blue;opacity:0.7;'></span>Vital Services GRFs</li>
    <li><span style='background:purple;opacity:0.7;'></span>Economic GRFs</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m

In [62]:
m.save("map_layer_selection2.html")